In [1]:
from taxcalc import *
from taxcalc.utils import *
from bokeh.io import show, output_notebook
from bokeh.charts import Bar, Scatter
from bokeh.layouts import column
from bokeh.plotting import figure
from collections import OrderedDict
import copy
from notebookfunctions import distribution, index_list, percentile
output_notebook()

Loading BokehJS ...

### Missing Variables

In [2]:
cps = pd.read_csv('../taxdata/cps_data/cps_ben_full.csv')
usable = open('taxcalc/records_variables.json')
usable_vars = json.load(usable)
usable.close()
missing = 0
for item in usable_vars['read'].keys():
    if item not in cps.columns:
        print '{}: {}'.format(item, usable_vars['read'][item]['desc'])
        missing += 1
print '\nTotal Variables Missing: {}'.format(missing)

p23250: Sch D: Net long-term capital gains/losses
p25470: Sch E: Royalty depletion and/or rental depreciation
e09800: Unreported payroll taxes from Form 4137 or 8919
e02000: Sch E rental, royalty, S-corp, etc, income/loss
e62900: Alternative Minimum Tax foreign tax credit from Form 6251
p08000: Other tax credits (but not including Sch R credit)
e58990: Investment income elected amount from Form 4952
e00700: Taxable refunds of state and local income taxes
e03290: Health savings account deduction from Form 8889
e07240: Retirement savings contributions credit from Form 8880
e19200: Sch A: Interest paid
e27200: Sch E: Farm rent net income or loss
e01200: Other net gain/loss from Form 4797
e03500: Alimony paid
e07260: Residential energy credit from Form 5695
p22250: Sch D: Net short-term capital gains/losses
e03220: Educator expenses
e07400: General business credit from Form 3800
f6251: 1 if Form 6251 (AMT) attached to return; otherwise 0
e03230: Tuition and fees from Form 8917
e03400: Pena

In [3]:
# Data from IRS-SOI Tax Stats. Used for comparison
soi_stats = pd.read_csv('soi_stats.csv', index_col=0)  # Equivalent to tax-calc diagnostic table
soi_income = pd.read_csv('soi_income_stats.csv')  # Distribution of income items
soi_deductions = pd.read_csv('soi_deductions.csv', index_col=0)  # Itemized deductions
soi_deductions['index'] = soi_deductions.index

In [4]:
# Create calculator from the PUF
calc_puf = Calculator(records=Records(), policy=Policy())
calc_puf.advance_to_year(2014)

You loaded data for 2009.
Tax-Calculator startup automatically extrapolated your data to 2013.


In [5]:
# Calculator the CPS
wt = pd.read_csv('../Dropbox/cps_weights.csv')
# wt.drop('SEQUENCE', inplace=True, axis=1)
# wt = wt.loc[:,:'WT2026']
wt *= 100
recs_cps = Records(data=cps,
                   weights=wt,
                   adjust_ratios=None,
                   start_year=2014)
pol_cps = Policy(start_year=2014, num_years=11)
calc_cps = Calculator(records=recs_cps, policy=pol_cps)
calc_cps.advance_to_year(2014)

In [6]:
calc_puf.calc_all()
calc_cps.calc_all()

### CPS Distribution Table

In [7]:
create_distribution_table(calc_cps.records, groupby='weighted_deciles', result_type='weighted_avg')

,expanded_income,s006,c00100,num_returns_StandardDed,standard,num_returns_ItemDed,c04470,c04600,c04800,taxbc,c62100,num_returns_AMT,c09600,c05800,c07100,othertaxes,refund,iitax,payrolltax,combined
0,-182,"16,318,429",-542,"9,212,955","6,107","14,925",7,"4,916",26,1,-547,0,0,1,0,0,90,-90,160,70
1,"8,802","16,320,544","4,791","11,461,211","7,374","155,301",93,"5,630",116,8,"4,755","17,179",1,8,2,0,718,-711,626,-85
2,"15,277","16,319,424","8,267","11,972,754","7,567","879,226",607,"6,004","1,315",124,"7,993","34,689",1,125,22,0,983,-881,"1,094",214
3,"21,865","16,319,273","14,491","12,924,656","7,570","2,041,912","1,266","6,657","4,069",414,"13,822","32,681",2,416,69,0,"1,173",-826,"1,969","1,143"
4,"29,317","16,319,617","21,952","12,693,373","7,585","3,122,581","1,962","7,199","8,603",982,"20,856","32,825",3,985,139,0,"1,022",-176,"3,025","2,849"
5,"38,837","16,318,693","30,962","12,373,953","7,665","3,758,391","2,620","7,626","15,245","1,817","29,596","25,077",4,"1,821",237,0,590,994,"4,277","5,272"
6,"51,862","16,320,896","44,180","11,716,124","7,463","4,533,853","3,701","8,015","25,900","3,266","42,326","30,054",3,"3,270",365,0,148,"2,757","6,029","8,786"
7,"71,353","16,318,688","64,672","10,828,694","7,180","5,484,529","5,081","8,792","43,899","6,218","62,418","28,158",4,"6,222",506,0,13,"5,703","8,843","14,546"
8,"103,024","16,320,295","94,946","8,161,910","5,773","8,156,266","7,924","9,903","71,451","10,880","91,934","46,130",7,"10,887",597,0,2,"10,288","13,090","23,378"
9,"245,340","16,319,911","231,860","4,918,038","3,643","11,401,342","14,692","9,765","203,807","46,844","228,039","2,942,303",628,"47,471",122,374,0,"47,723","21,897","69,621"


### PUF Distribution Table

In [8]:
create_distribution_table(calc_puf.records, groupby='weighted_deciles', result_type='weighted_avg')

,expanded_income,s006,c00100,num_returns_StandardDed,standard,num_returns_ItemDed,c04470,c04600,c04800,taxbc,c62100,num_returns_AMT,c09600,c05800,c07100,othertaxes,refund,iitax,payrolltax,combined
0,"-6,904","16,222,114","-7,252","11,094,216","5,964","6,150",18,"4,135",44,4,"-7,601",0,0,4,0,6,141,-131,353,222
1,"8,443","16,220,773","6,016","13,627,903","7,057","48,948",29,"4,913",262,23,"5,990","20,950",0,24,0,4,774,-747,821,74
2,"15,025","16,223,499","10,098","13,243,287","7,608","279,753",180,"6,493","1,482",145,"9,974","27,189",1,146,19,6,"1,650","-1,517","1,428",-89
3,"21,963","16,222,202","15,120","14,169,419","7,787","683,754",548,"6,822","3,991",420,"14,767","31,485",2,422,82,13,"1,460","-1,107","2,039",932
4,"30,413","16,223,284","22,440","14,142,144","7,921","1,516,658","1,287","7,187","8,668",999,"21,647","18,616",2,"1,002",217,17,"1,067",-265,"2,881","2,615"
5,"40,936","16,222,555","32,564","13,206,631","7,687","2,863,096","2,661","7,339","16,532","2,017","30,986","15,840",2,"2,020",362,26,548,"1,136","3,965","5,101"
6,"54,706","16,222,223","46,478","11,339,850","7,067","4,849,894","5,036","7,734","27,243","3,573","43,467","10,617",2,"3,575",506,42,164,"2,947","5,336","8,283"
7,"74,668","16,222,702","67,782","9,028,824","6,082","7,185,021","8,156","8,574","45,098","6,519","63,224","46,267",5,"6,524",681,62,80,"5,825","7,708","13,533"
8,"110,558","16,222,104","102,839","5,871,767","4,308","10,349,258","13,730","10,142","74,728","11,702","95,552","184,541",18,"11,720",889,100,95,"10,835","12,479","23,314"
9,"338,763","16,223,164","321,658","2,019,136","1,497","14,190,613","36,479","9,580","274,512","69,165","306,721","4,448,053","2,351","71,516","1,789","2,138",55,"71,811","21,957","93,768"


### Diagnostic Table Comparison

In [9]:
cps_diag = create_diagnostic_table(calc_cps)

In [10]:
puf_diag = create_diagnostic_table(calc_puf)

In [11]:
diag_data = pd.DataFrame()
diag_data['SOI'] = soi_stats['Value']
diag_data['CPS'] = cps_diag[2014]
diag_data['PUF'] = puf_diag[2014]
diag_data['% Change'] = ((cps_diag[2014] / puf_diag[2014]) - 1) * 100

In [12]:
diag_data

,SOI,CPS,PUF,% Change
Returns (#m),148.6,163.2,162.2,0.6
AGI ($b),"9,771.00","8,414.2","10,021.5",-16.0
Itemizers (#m),44,39.5,42.0,-5.8
Itemized Deduction ($b),1206.7,619.4,"1,105.2",-44.0
Standard Deduction Filers (#m),117.4,106.3,107.7,-1.4
Standard Deduction ($b),876.2,965.8,925.2,4.4
Personal Exemption ($b),1121.6,"1,110.2","1,114.0",-0.3
Taxable Income ($b),6997.9,"6,110.6","7,341.8",-16.8
Regular Tax ($b),NaN,"1,151.4","1,534.2",-24.9
AMT Income ($b),NaN,"8,179.4","9,485.9",-13.8


In [13]:
# Total value of missing itemized deductions
in_billions = 1e-9
state = (calc_puf.records.e18400 * calc_puf.records.s006).sum()
print 'State and Local: {} ($b)'.format(round(state * in_billions, 2))
int_paid = (calc_puf.records.e19200 * calc_puf.records.s006).sum()
print 'Interest Paid: {} ($b)'.format(round(int_paid * in_billions, 2))
net_cas = (calc_puf.records.g20500 * calc_puf.records.s006).sum()
print 'Net Casualty or Theft Loss: {} ($b)'.format(round(net_cas * in_billions, 2))
print '-----------------'
print 'Total: {} ($b)'.format((round((state + int_paid + net_cas) * in_billions, 2)))

State and Local: 321.94 ($b)
Interest Paid: 325.15 ($b)
Net Casualty or Theft Loss: 4.62 ($b)
-----------------
Total: 651.72 ($b)


### Income Levels

In [14]:
inc_dict = OrderedDict()
inc_dict['CPS'] = [] 
inc_dict['PUF'] = []
inc_dict['Diff'] = []
inc_dict['Pct Diff'] = []
inc_list = ['WAS', 'Taxable Interest', 'Ordinary Dividends', 'Qualified Dividends', 'Business Income']
was_cps = (calc_cps.records.e00200 * calc_cps.records.s006).sum()
inc_dict['CPS'].append(was_cps)
was_puf = (calc_puf.records.e00200 * calc_puf.records.s006).sum()
inc_dict['PUF'].append(was_puf)
was_diff = (was_cps - was_puf)
inc_dict['Diff'].append(was_diff)
inc_dict['Pct Diff'].append((was_diff / was_puf) * 100)

int_cps = (calc_cps.records.e00300 * calc_cps.records.s006).sum()
inc_dict['CPS'].append(int_cps)
int_puf = (calc_puf.records.e00300 * calc_puf.records.s006).sum()
inc_dict['PUF'].append(int_puf)
int_diff = (int_cps - int_puf)
inc_dict['Diff'].append(int_diff)
inc_dict['Pct Diff'].append((int_diff / int_puf) * 100)

odiv_cps = (calc_cps.records.e00600 * calc_cps.records.s006).sum()
inc_dict['CPS'].append(odiv_cps)
odiv_puf = (calc_puf.records.e00600 * calc_puf.records.s006).sum()
inc_dict['PUF'].append(odiv_puf)
odiv_diff = (odiv_cps - odiv_puf)
inc_dict['Diff'].append(odiv_diff)
inc_dict['Pct Diff'].append((odiv_diff / odiv_puf) * 100)

qdiv_cps = (calc_cps.records.e00650 * calc_cps.records.s006).sum()
inc_dict['CPS'].append(qdiv_cps)
qdiv_puf = (calc_puf.records.e00650 * calc_puf.records.s006).sum()
inc_dict['PUF'].append(qdiv_puf)
qdiv_diff = (qdiv_cps - qdiv_puf)
inc_dict['Diff'].append(qdiv_diff)
inc_dict['Pct Diff'].append((qdiv_diff / qdiv_puf) * 100)

biz_cps = (calc_cps.records.e00900 * calc_cps.records.s006).sum()
inc_dict['CPS'].append(biz_cps)
biz_puf = (calc_puf.records.e00900 * calc_puf.records.s006).sum()
inc_dict['PUF'].append(biz_puf)
biz_diff = (biz_cps - biz_puf)
inc_dict['Diff'].append(biz_diff)
inc_dict['Pct Diff'].append((biz_diff / biz_puf) * 100)

inc_df = pd.DataFrame.from_dict(inc_dict)
inc_df.index = inc_list
inc_df

,CPS,PUF,Diff,Pct Diff
WAS,"6,648,838,107,939.7","6,820,912,371,695.9","-172,074,263,756.2",-2.5
Taxable Interest,"91,960,641,825.4","97,246,196,658.4","-5,285,554,833.1",-5.4
Ordinary Dividends,"245,424,347,487.0","270,137,086,776.2","-24,712,739,289.2",-9.1
Qualified Dividends,"185,442,636,961.2","198,124,994,913.5","-12,682,357,952.3",-6.4
Business Income,"309,555,316,495.2","309,444,223,964.1","111,092,531.1",0.0


### Distribution of Income Variables

In [15]:
# Generate data for distribution plots
cps_dist = pd.DataFrame()
puf_dist = pd.DataFrame()
cps_was = distribution(calc_cps.records.e00200, calc_cps.records.s006, calc_cps.records.c00100)
puf_was = distribution(calc_puf.records.e00200, calc_puf.records.s006, calc_puf.records.c00100)
cps_int = distribution(calc_cps.records.e00300, calc_cps.records.s006, calc_cps.records.c00100)
puf_int = distribution(calc_puf.records.e00300, calc_puf.records.s006, calc_puf.records.c00100)
cps_odiv = distribution(calc_cps.records.e00600, calc_cps.records.s006, calc_cps.records.c00100)
puf_odiv = distribution(calc_puf.records.e00600, calc_puf.records.s006, calc_puf.records.c00100)
cps_qdiv = distribution(calc_cps.records.e00650, calc_cps.records.s006, calc_cps.records.c00100)
puf_qdiv = distribution(calc_puf.records.e00650, calc_puf.records.s006, calc_puf.records.c00100)
cps_biz = distribution(calc_cps.records.e00900, calc_cps.records.s006, calc_cps.records.c00100)
puf_biz = distribution(calc_puf.records.e00900, calc_puf.records.s006, calc_puf.records.c00100)
cps_dist['WAS'] = cps_was[1]
puf_dist['WAS'] = puf_was[1]
cps_dist['INT'] = cps_int[1]
puf_dist['INT'] = puf_int[1]
cps_dist['ODIV'] = cps_odiv[1]
puf_dist['ODIV'] = puf_odiv[1]
cps_dist['QDIV'] = cps_qdiv[1]
puf_dist['QDIV'] = puf_qdiv[1]
cps_dist['BIZ'] = cps_biz[1]
puf_dist['BIZ'] = puf_biz[1]
cps_dist['AGI Bin'] = index_list()
puf_dist['AGI Bin'] = index_list()
cps_dist['label'] = 'CPS'
puf_dist['label'] = 'PUF'
# Create scatter plot objects
items_tups = [('WAS', 'WAS'), ('INT', 'Interest Income'), ('ODIV', 'Ordinary Dividends'),
              ('QDIV', 'Qualified Dividends'), ('BIZ', 'Business Income')]
soi_dist = pd.DataFrame()
for item in items_tups:
    soi_dist[item[0]] = (soi_income[item[0]] / soi_income[item[0]].sum()) * 100
soi_dist['AGI Bin'] = index_list()
soi_dist['label'] = 'SOI'
scatter_data = pd.concat([cps_dist, puf_dist, soi_dist])
scatter_list = list()  # list for scatter plot objects
for item in items_tups:
    title = 'Percent of Total {} by AGI Bin'.format(item[1])
    scatter = Scatter(scatter_data, x='AGI Bin', y=item[0], color='label', ylabel='Percent',
                      title=title, tooltips=[('PCT', '@{}'.format(item[0]))])
    scatter_list.append(scatter)

In [16]:
show(column(scatter_list))

In [17]:
cps_tot = pd.DataFrame()
puf_tot = pd.DataFrame()
cps_tot['WAS'] = cps_was[0]
puf_tot['WAS'] = puf_was[0]
cps_tot['INT'] = cps_int[0]
puf_tot['INT'] = puf_int[0]
cps_tot['ODIV'] = cps_odiv[0]
puf_tot['ODIV'] = puf_odiv[0]
cps_tot['QDIV'] = cps_qdiv[0]
puf_tot['QDIV'] = puf_qdiv[0]
cps_tot['BIZ'] = cps_biz[0]
puf_tot['BIZ'] = puf_biz[0]
cps_tot['AGI Bin'] = index_list()
puf_tot['AGI Bin'] = index_list()
cps_tot['label'] = 'CPS'
puf_tot['label'] = 'PUF'
soi_income['AGI Bin'] = index_list()
soi_income['label'] = 'SOI'
total_data = pd.concat([cps_tot, puf_tot, soi_income])
# Create scatter plot objects
items_tups = [('WAS', 'WAS'), ('INT', 'Interest Income'), ('ODIV', 'Ordinary Dividends'),
              ('QDIV', 'Qualified Dividends'), ('BIZ', 'Business Income')]
total_list = list()  # list for scatter plot objects
for item in items_tups:
    title = 'Total {} by AGI Bin'.format(item[1])
    scatter = Scatter(total_data, x='AGI Bin', y=item[0], color='label', ylabel='Total',
                      title=title, tooltips=[('Total', '@{}'.format(item[0]))])
    total_list.append(scatter)

In [18]:
show(column(total_list))

### Itemized Deduction Amounts

In [19]:
deductions_cps = {'Medical Expenses': (calc_cps.records.e17500[calc_cps.records.c04470 > 0] *
                                       calc_cps.records.s006[calc_cps.records.c04470 > 0]).sum(),
                  'State and Local Taxes':  (calc_cps.records.e18400[calc_cps.records.c04470 > 0] *
                                             calc_cps.records.s006[calc_cps.records.c04470 > 0]).sum(),
                  'Real Estate Taxes':  (calc_cps.records.e18500[calc_cps.records.c04470 > 0] *
                                         calc_cps.records.s006[calc_cps.records.c04470 > 0]).sum(),
                  'Interest Paid':  (calc_cps.records.e19200[calc_cps.records.c04470 > 0] *
                                     calc_cps.records.s006[calc_cps.records.c04470 > 0]).sum(),
                  'Charitable Cash Contributions': (calc_cps.records.e19800[calc_cps.records.c04470 > 0] *
                                                    calc_cps.records.s006[calc_cps.records.c04470 > 0]).sum(),
                  'Charitable Non-Cash Contributions': (calc_cps.records.e20100[calc_cps.records.c04470 > 0] *
                                                        calc_cps.records.s006[calc_cps.records.c04470 > 0]).sum(),
                  'Total Misc. Expenses':  (calc_cps.records.e20400[calc_cps.records.c04470 > 0] *
                                            calc_cps.records.s006[calc_cps.records.c04470 > 0]).sum(),
                  'Net Casualty or Loss': (calc_cps.records.g20500[calc_cps.records.c04470 > 0] *
                                           calc_cps.records.s006[calc_cps.records.c04470 > 0]).sum()}
ded_cps_df = pd.DataFrame.from_dict(deductions_cps, 'index')
ded_cps_df.columns = ['Total']
ded_cps_df['source'] = 'CPS'

deductions_puf = {'Medical Expenses': (calc_puf.records.e17500[calc_puf.records.c04470 > 0] *
                                       calc_puf.records.s006[calc_puf.records.c04470 > 0]).sum(),
                  'State and Local Taxes':  (calc_puf.records.e18400[calc_puf.records.c04470 > 0] *
                                             calc_puf.records.s006[calc_puf.records.c04470 > 0]).sum(),
                  'Real Estate Taxes':  (calc_puf.records.e18500[calc_puf.records.c04470 > 0] *
                                         calc_puf.records.s006[calc_puf.records.c04470 > 0]).sum(),
                  'Interest Paid':  (calc_puf.records.e19200[calc_puf.records.c04470 > 0] *
                                     calc_puf.records.s006[calc_puf.records.c04470 > 0]).sum(),
                  'Charitable Cash Contributions': (calc_puf.records.e19800[calc_puf.records.c04470 > 0] *
                                                    calc_puf.records.s006[calc_puf.records.c04470 > 0]).sum(),
                  'Charitable Non-Cash Contributions': (calc_puf.records.e20100[calc_puf.records.c04470 > 0] *
                                                        calc_puf.records.s006[calc_puf.records.c04470 > 0]).sum(),
                  'Total Misc. Expenses':  (calc_puf.records.e20400[calc_puf.records.c04470 > 0] *
                                            calc_puf.records.s006[calc_puf.records.c04470 > 0]).sum(),
                  'Net Casualty or Loss': (calc_puf.records.g20500[calc_puf.records.c04470 > 0] *
                                           calc_puf.records.s006[calc_puf.records.c04470 > 0]).sum()}
ded_puf_df = pd.DataFrame.from_dict(deductions_puf, 'index')
ded_puf_df.columns = ['Total']
ded_puf_df['source'] = 'PUF'
soi_deductions['source'] = 'SOI'

ded_full_df = pd.concat([ded_cps_df, ded_puf_df, soi_deductions])
ded_full_df['index'] = ded_full_df.index

In [20]:
ded_bar = Bar(ded_full_df, 'index', 'Total', group='source', title='Itemized Deduction Totals',
              xlabel='Deduction', ylabel='Total', tooltips=[('Deduction', '@index'), ('Total', '@height'),
                                                            ('Data', '@source')])
show(ded_bar)

In [21]:
ded_error_df = pd.DataFrame()
ded_error_df['Difference - CPS'] = ded_cps_df['Total'] - soi_deductions['Total']
ded_error_df['% Difference - CPS'] = 100 * ded_error_df['Difference - CPS'] / soi_deductions['Total']
ded_error_df['Difference - PUF'] = ded_puf_df['Total'] - soi_deductions['Total']
ded_error_df['% Difference - PUF'] = 100 * ded_error_df['Difference - PUF'] / soi_deductions['Total']
print 'Error in Itemized Deductions Relative to SOI Totals'
ded_error_df

Error in Itemized Deductions Relative to SOI Totals


,Difference - CPS,% Difference - CPS,Difference - PUF,% Difference - PUF
Charitable Cash Contributions,"-15,578,764,104.2",-10.0,"-4,703,427,583.1",-3.0
Charitable Non-Cash Contributions,"-34,652,972,796.2",-53.0,"-31,555,102,263.5",-48.3
Interest Paid,"-307,962,385,000.0",-100.0,"-7,489,448,194.8",-2.4
Medical Expenses,"-22,933,690,785.2",-17.8,"-9,779,156,210.6",-7.6
Net Casualty or Loss,"-2,204,349,000.0",-100.0,"2,251,393,716.6",102.1
Real Estate Taxes,"-103,310,686,116.8",-57.1,"17,247,882,146.1",9.5
State and Local Taxes,"-92,296,162,578.9",-28.1,"-16,248,434,290.2",-4.9
Total Misc. Expenses,"-25,602,945,716.1",-20.3,"3,957,677,605.4",3.1


### Refundable Credits

In [22]:
eitc_cps = (calc_cps.records.eitc * calc_cps.records.s006).sum()
c11070_cps = (calc_cps.records.c11070 * calc_cps.records.s006).sum()
c10960_cps = (calc_cps.records.c10960 * calc_cps.records.s006).sum()
personal_credit_cps = (calc_cps.records.personal_credit * calc_cps.records.s006).sum()
ctc_new_cps = (calc_cps.records.ctc_new * calc_cps.records.s006).sum()

eitc_puf = (calc_puf.records.eitc * calc_puf.records.s006).sum()
c11070_puf = (calc_puf.records.c11070 * calc_puf.records.s006).sum()
c10960_puf = (calc_puf.records.c10960 * calc_puf.records.s006).sum()
personal_credit_puf = (calc_puf.records.personal_credit * calc_puf.records.s006).sum()
ctc_new_puf = (calc_puf.records.ctc_new * calc_puf.records.s006).sum()

In [23]:
print 'Refundable Credits'
pd.DataFrame(OrderedDict({'PUF': [eitc_puf, c11070_puf, c10960_puf, personal_credit_puf, ctc_new_puf],
                          'CPS': [eitc_cps, c11070_cps, c10960_cps, personal_credit_cps, ctc_new_cps],
                          'Diff': [eitc_cps - eitc_puf, c11070_cps - c11070_puf, c10960_cps - c10960_puf,
                                   personal_credit_cps - personal_credit_puf,
                                   ctc_new_cps - ctc_new_puf]}),
                          index=['eitc', 'c11070', 'c10960', 'personal credit', 'ctc new'])

Refundable Credits


,Diff,PUF,CPS
eitc,"-6,867,495,315.7","65,207,453,506.9","58,339,958,191.2"
c11070,"-5,271,114,412.6","24,274,900,731.4","19,003,786,318.8"
c10960,"-8,382,574,969.7","8,382,574,969.7",0.0
personal credit,0.0,0.0,0.0
ctc new,0.0,0.0,0.0


In [24]:
eic0cps = sum(calc_cps.records.s006[calc_cps.records.EIC == 0])
eic1cps = sum(calc_cps.records.s006[calc_cps.records.EIC == 1])
eic2cps = sum(calc_cps.records.s006[calc_cps.records.EIC == 2])
eic3cps = sum(calc_cps.records.s006[calc_cps.records.EIC == 3])
eic0puf = sum(calc_puf.records.s006[calc_puf.records.EIC == 0])
eic1puf = sum(calc_puf.records.s006[calc_puf.records.EIC == 1])
eic2puf = sum(calc_puf.records.s006[calc_puf.records.EIC == 2])
eic3puf = sum(calc_puf.records.s006[calc_puf.records.EIC == 3]) 
eic_tot_cps = calc_cps.records.s006.sum()
eic_tot_puf = calc_puf.records.s006.sum()
print 'Percent and Number of Tax Units with Specified Number of EIC Qualified Children'
pd.DataFrame({'CPS': [eic0cps, eic1cps, eic2cps, eic3cps],
              'PUF': [eic0puf, eic1puf, eic2puf, eic3puf],
              '% - CPS': [eic0cps / eic_tot_cps, eic1cps / eic_tot_cps,
                          eic2cps / eic_tot_cps, eic3cps / eic_tot_cps],
              '% - PUF': [eic0puf / eic_tot_puf, eic1puf / eic_tot_puf,
                          eic2puf / eic_tot_puf, eic3puf / eic_tot_puf]})

Percent and Number of Tax Units with Specified Number of EIC Qualified Children


,% - CPS,% - PUF,CPS,PUF
0,0.7,0.9,"114,745,569.5","140,952,059.1"
1,0.1,0.1,"22,983,420.2","10,061,670.4"
2,0.1,0.0,"16,819,448.2","7,931,481.3"
3,0.1,0.0,"8,647,332.5","3,279,408.9"


### Benefit Programs

In [25]:
# Columns used for participation rates
cps['ssi_part'] = np.where(cps.ssi > 0, 1, 0)
cps['snap_part'] = np.where(cps.snap > 0, 1, 0)
cps['mcare_part'] = np.where(cps.mcare > 0, 1, 0)
cps['mcaid_part'] = np.where(cps.mcaid > 0, 1, 0)
cps['ss_part'] = np.where(cps.ss > 0, 1, 0)
cps['vb_part'] = np.where(cps.vb > 0, 1, 0)

In [26]:
benefits = pd.DataFrame({'SSI': [(cps.ssi * cps.s006).sum()],
                         'SNAP': [(cps.snap * cps.s006).sum()],
                         'Medicare': [(cps.mcare * cps.s006).sum()],
                         'Medicaid': [(cps.mcaid * cps.s006).sum()],
                         'Social Security': [(cps.ss * cps.s006).sum()],
                         'VB': [(cps.vb * cps.s006).sum()]}).transpose()
benefits.columns = ['Total']
print 'Benefits Totals'
benefits

Benefits Totals


,Total
Medicaid,"352,985,014,221.6"
Medicare,"488,236,708,639.1"
SNAP,"82,336,140,621.7"
SSI,"54,186,915,454.5"
Social Security,"630,068,059,192.4"
VB,"146,836,134,085.4"


#### Benefit Program Participation Rates

In [27]:
benefit_participation_names = [('ssi_part', 'SSI'), ('snap_part', 'SNAP'), ('mcare_part', 'Medicare'),
                               ('mcaid_part', 'Medicaid'), ('ss_part', 'Social Security'),
                               ('vb_part', "Veteran's Benefits")]

In [28]:
# Create list of bokeh figure displaying participation rates in each program
fig_list = list()
for item in benefit_participation_names:
    wcps = percentile(cps, item[0], 100, 'e00200', 's006')
    f = figure(title='{} Participation Rate'.format(item[1]),
               x_axis_label='Wage Percentile')
    f.line(wcps.index, wcps)
    fig_list.append(f)

In [29]:
show(column(fig_list))

#### Average Benefits Received by Participants

In [30]:
benefit_program_names = [('ssi', 'SSI'), ('snap', 'SNAP'), ('mcare', 'Medicare'),
                         ('mcaid', 'Medicaid'), ('ss', 'Social Security'),
                         ('vb', "Veteran's Benefits")]

In [35]:
fig_list = list()
for item in benefit_program_names:
    # Only taking the average benefit of participants
    bcps = cps[cps[item[0]] > 0]
    wcps = percentile(bcps, item[0], 100, 'e00200', 's006')
    f = figure(title='Average {} Benefit - Participants'.format(item[1]),
               x_axis_label='Wage Percentile')
    f.line(wcps.index, wcps)
    fig_list.append(f)

In [36]:
show(column(fig_list))

#### Average Benefits Received by Entire Population

In [33]:
fig_list = list()
for item in benefit_program_names:
    # Only taking the average benefit of participants
    wcps = percentile(cps, item[0], 100, 'e00200', 's006')
    f = figure(title='Average {} Benefit - Entire Population'.format(item[1]),
               x_axis_label='Wage Percentile')
    f.line(wcps.index, wcps)
    fig_list.append(f)

In [34]:
show(column(fig_list))